In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/rank_64/rank_16_2/checkpoint-50/"
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True, device_map="auto")

/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
def generate_content(tokenizer, messages):
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    tokenizer.pad_token = tokenizer.eos_token
    with torch.no_grad():
        outputs = model.generate(
                input_ids=input_ids,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
        )
        
        val = (tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1])
    
    return val, input_ids

In [3]:
import pandas as pd

df = pd.read_csv('../../data/processed_data.csv')


from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(df, df['scene']):
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    


In [4]:
import json
import ast
X_test['content'] = X_test['content'].apply(lambda x: ast.literal_eval(x))

/tmp/ipykernel_122764/2504390441.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['content'] = X_test['content'].apply(lambda x: ast.literal_eval(x))


In [5]:
X_train['content'] = X_train['content'].apply(lambda x: x[1:])
X_test['content'] = X_test['content'].apply(lambda x: x[1:])

/tmp/ipykernel_122764/778610680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['content'] = X_train['content'].apply(lambda x: x[1:])
/tmp/ipykernel_122764/778610680.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['content'] = X_test['content'].apply(lambda x: x[1:])


In [51]:
test_data = X_test['content'].iloc[0:100].to_list()

In [7]:
print(test_data[0])

[{'role': 'user', 'content': 'Hi there, how are you?'}, {'role': 'assistant', 'content': "Hello! I'm here to listen. How are you doing today?"}, {'role': 'user', 'content': "I've been feeling really down. I lost my beloved pet dog recently, and I can't seem to shake off this overwhelming sadness and loneliness."}, {'role': 'assistant', 'content': "I'm so sorry to hear about the loss of your furry friend. It's completely normal to feel devastated and lonely after such an emotional event."}, {'role': 'user', 'content': "Thank you. It's just been really hard adjusting to life without them. They were my constant companion and provided so much comfort."}, {'role': 'assistant', 'content': 'Losing a pet can be like losing a family member, and it takes time to grieve and adapt to the void they leave behind.'}, {'role': 'user', 'content': "Yes, exactly. I find myself missing their presence all the time and it's harder than I thought it would be."}, {'role': 'assistant', 'content': 'I can only i

In [52]:
import spacy

nlp = spacy.load('en_core_web_sm')

included_tags = {"NOUN", "VERB", "INTJ" "ADJ", "ADV", "ADP", "PROPN", "NUM", "SCONJ", "PRON"}
def get_sentence(sentence):
    new_sentence = []
    for token in nlp(sentence):
        #print(token.pos_)
        if token.pos_ in included_tags:
            new_sentence.append(token.text)
    res = " ".join(new_sentence)
    return res

get_sentence(test_data[0][0]['content'])


'there how you'

In [53]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    #print("stopwords")
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [54]:
from tqdm import tqdm

max_length = 1024
stride = 128

#encodings = tokenizer.encode_plus(data['dialog'][1]['text'], return_tensors="pt").to('cuda')
tokenizer.pad_token = tokenizer.eos_token
seq_len = 1024

In [55]:
messages_final = []
messages_no_sw_final = []
messages_boh_final = []
messages_no_sw_original_final = []
messages_boh_original_final = []

perplexity_final, perplexity_no_sw_final, perplexity_boh_final = [], [], []
nlls_final, nlls_no_sw_final, nlls_boh_final = [], [], []
prev_end_loc = 0
for data in test_data:
    print(data)
    messages = []
    messages_no_sw = []
    messages_no_sw_original = []
    messages_boh = []
    messages_boh_original = []
    
    nlls, nlls_boh, nlls_no_sw = [], [], []
    
    perplexity, perplexity_no_sw, perplexity_boh = [], [], []
    try:
        for i in range(0, len(data), 2):
            for begin_loc in (range(0, seq_len, stride)):
                #print(type(data[i]['content']))
                messages.append({"role": "user", "content":  data[i]['content']})
                messages_no_sw.append({"role": "user", "content": remove_stopwords(data[i]['content'])})
                messages_no_sw_original.append({"role": "user", "content": data[i]['content']})
                messages_boh.append({"role": "user", "content": get_sentence(data[i]['content'])})
                messages_boh_original.append({"role": "user", "content": data[i]['content']})
                
                i+=2
                
                end_loc = min(begin_loc + max_length, seq_len)
                trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
                #input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
                input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
                input_ids_no_sw = tokenizer.apply_chat_template(messages_no_sw, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
                input_ids_boh = tokenizer.apply_chat_template(messages_boh, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
                target_ids = input_ids.clone()
                target_ids_boh = input_ids_boh.clone()
                target_ids_no_sw = input_ids_no_sw.clone()
                target_ids[:, :-trg_len] = -100
                target_ids_boh[:, :-trg_len] = -100
                target_ids_no_sw[:, :-trg_len] = -100
                
                with torch.no_grad():
                    val, _ = generate_content(tokenizer, messages)
                    val_no_sw, _ = generate_content(tokenizer, messages_no_sw)
                    val_boh, _ = generate_content(tokenizer, messages_boh)
                    #print(val)
                    outputs = model(input_ids=input_ids, labels=target_ids)
                    outputs_boh = model(input_ids=input_ids_boh, labels=target_ids_boh)
                    outputs_no_sw = model(input_ids=input_ids_no_sw, labels=target_ids_no_sw)
                    
                    messages.append({"role": "assistant", "content": val})
                    messages_no_sw.append({"role": "assistant", "content": remove_stopwords(val_no_sw)})
                    messages_no_sw_original.append({"role": "assistant", "content": val_no_sw})
                    messages_boh.append({"role": "assistant", "content": get_sentence(val_boh)})
                    messages_boh_original.append({"role": "assistant", "content": val_boh})
                    
                    neg_log_likelihood = outputs.loss
                    neg_log_likelihood_boh = outputs_boh.loss
                    neg_log_likelihood_no_sw = outputs_no_sw.loss
                    
                nlls.append(neg_log_likelihood)
                nlls_boh.append(neg_log_likelihood_boh)
                nlls_no_sw.append(neg_log_likelihood_no_sw)
                
                prev_end_loc = end_loc
                if end_loc == seq_len:
                    break
                    
            ppl = torch.exp(torch.stack(nlls).mean())
            ppl_boh = torch.exp(torch.stack(nlls_boh).mean())
            ppl_no_sw = torch.exp(torch.stack(nlls_no_sw).mean())
            print(ppl)
            print(ppl_boh)
            print(ppl_no_sw)
            perplexity.append(ppl)
            perplexity_boh.append(ppl_boh)
            perplexity_no_sw.append(ppl_no_sw)
            
    except Exception as e:
        print(e)
        pass
        
    messages_final.append(messages)
    messages_no_sw_final.append(messages_no_sw)
    messages_no_sw_original_final.append(messages_no_sw_original)
    messages_boh_final.append(messages_boh)
    messages_boh_original_final.append(messages_boh_original)
    nlls_final.append(nlls)
    nlls_no_sw_final.append(nlls_no_sw)
    nlls_boh_final.append(nlls_boh)
    perplexity_final.append(perplexity)
    perplexity_no_sw_final.append(perplexity_no_sw)
    perplexity_boh_final.append(perplexity_boh)
                
            
            

[{'role': 'user', 'content': 'Hi there, how are you?'}, {'role': 'assistant', 'content': "Hello! I'm here to listen. How are you doing today?"}, {'role': 'user', 'content': "I've been feeling really down. I lost my beloved pet dog recently, and I can't seem to shake off this overwhelming sadness and loneliness."}, {'role': 'assistant', 'content': "I'm so sorry to hear about the loss of your furry friend. It's completely normal to feel devastated and lonely after such an emotional event."}, {'role': 'user', 'content': "Thank you. It's just been really hard adjusting to life without them. They were my constant companion and provided so much comfort."}, {'role': 'assistant', 'content': 'Losing a pet can be like losing a family member, and it takes time to grieve and adapt to the void they leave behind.'}, {'role': 'user', 'content': "Yes, exactly. I find myself missing their presence all the time and it's harder than I thought it would be."}, {'role': 'assistant', 'content': 'I can only i

In [58]:
with open ('messages_final.json', 'w') as f:
    json.dump(messages_final, f)

In [59]:
with open ('messages_no_sw_final.json', 'w') as f:
    json.dump(messages_no_sw_final, f)

In [60]:
with open ('messages_no_sw_original_final.json', 'w') as f:
    json.dump(messages_no_sw_original_final, f)

In [61]:
with open ('messages_boh_final.json', 'w') as f:
    json.dump(messages_boh_final, f)

In [62]:
with open ('messages_boh_original_final.json', 'w') as f:
    json.dump(messages_boh_original_final, f)

In [83]:
with open('test_data.json', 'w') as f:
    json.dump(test_data, f)

In [64]:
m = json.loads(open('messages_final.json').read())
print(len(m))

100


In [71]:
torch.save(nlls_final, 'nlls_final.pt')
torch.save(nlls_no_sw_final, 'nlls_no_sw_final.pt')
torch.save(nlls_boh_final, 'nlls_boh_final.pt')
torch.save(perplexity_final, 'perplexity_final.pt')
torch.save(perplexity_no_sw_final, 'perplexity_no_sw_final.pt')
torch.save(perplexity_boh_final, 'perplexity_boh_final.pt')


In [ ]:
perplexity_final

In [ ]:
t = torch.load('nlls_final.pt')
t